In [10]:
import sys
import pygame
import random
import pygame_menu
import pygame.freetype
import pygame.image as pyimg
import numpy as np
sys.path.append(r"C:/Users/chris/OneDrive/Documents/Masters Project")
from ipynb.fs.full.Dependencies.Training_Game import Training_Game

import os
os.chdir(r"C:/Users/chris/OneDrive/Documents/Masters Project/")
pygame.init()

from io import StringIO 
from PIL import Image

In [11]:
#Core Space invaders game mode.
class Space_Invaders():
    def __init__(self, width, height, screen, surface, mode):
        #Pygame initialisation variables
        self.GAME_FONT = pygame.freetype.Font("Dependencies/Resources/COMIC.TTF", 24)
        self.width = width
        self.height = height
        self.screen = pygame.display.set_mode((width, height), pygame.DOUBLEBUF)
        
        self.clock = pygame.time.Clock()
        
        #Game state check
        self.done = False
        self.lost = False
        self.training_mode = mode
        
        self.scale = 1
        self.enemySpeed = 0.125
        self.movement_speed = 2
        self.rocket_speed = 10
        self.enemy_rocket_speed = -3
        
        if self.training_mode == True:
            self.enemySpeed = 0.083
            self.movement_speed = 1
            self.rocket_speed = 6
            self.enemy_rocket_speed = -1.25
            
        #Enemy general variables
        self.enemyShotTimer = random.uniform(0.5, 10.0)
        self.enemyRockets = []
        self.rockets = []
        self.potentialShooters = []
        self.aliens = []
        self.generator = Generator(self)
        self.max_aliens = len(self.aliens)
        
        #Misc
        player_size = 30
            
        self.player = Invaders_Player(self, width/2, height-(height/100 * 10), player_size)
        print("player position:", width/2, height-(height/100 * 10))
        self.all_sprites_list = pygame.sprite.Group()
        
        self.current_reward = 0
        self.render_delay = 10
        
        #If in a normal game, start the standard game loop, otherwise this will be controlled
        #by the space invader env super-class.
        if self.training_mode == False:
            while not self.done:
                self.update()
                #pygame.display.flip()
                #self.clock.tick(60)
                self.render()

            print("exitng")
            pygame.display.quit()
            pygame.quit()
            sys.exit()
            
    def execute_action(self, action):
        if action == 0:
            self.move_left()
        if action == 1:
            self.move_right()
        if action == 2:
            self.shoot()
                
    def move_left(self):
        self.player.sprite.rect.x -=self.movement_speed if \
        self.player.sprite.rect.x > 0 else 0

    def move_right(self):
        if self.player.sprite.rect.x < self.width - 30:
            self.player.sprite.rect.x +=self.movement_speed

    def shoot(self):
        origin_point = 16
        #if self.training_mode == True:
        #    origin_point = 1.6
        if self.player.fire_rate <= 0.0:
            print("successfully shot")
            self.rockets.append(Rocket(self, self.player.sprite.rect.x+origin_point,
                                       self.player.sprite.rect.y))
            self.player.fire_rate = 3.0

    def get_state(self):
            data = pygame.image.tostring(self.screen, 'RGB')
            img = Image.frombytes('RGB', (self.width, self.height), data)
            #img.save('before.png')
            img = img.convert('L')
            #img.save('after.png')
            #print("saved")
           # print("object type: ", type(img.getdata()))
            #print(np.array(img.getdata()).shape)
            #print(np.array(img.getdata()))
            return np.array(img.getdata())
    
    def take_picture(self):
        data = pygame.image.tostring(self.screen, 'RGB')
        img = Image.frombytes('RGB', (self.width, self.height), data)
        #img.save('before.png')
        img = img.convert('L')
        img.save('after.png')
        print("saved")
       # print("object type: ", type(img.getdata()))
        print(np.array(img.getdata()).shape)
        print(np.array(img.getdata()))
        return img
        

    def calculate_reward(self):
        #Return 1 or -1 to indicate a loss or win terminal state
        if self.lost == True:
            return 0
        if self.done and not self.lost or len(self.aliens) == 0:
            return 1
        
        reward = self.current_reward
        self.current_reward = 0
        return reward
        #If still alive, calculate a function, clamped between 0-0.01 giving small reward,
        #scaled upon how many aliens are dead in the state to hopefully promote both preserving
        #its own life and killing other aliens.
        #aliens_remaining = len(self.aliens)
        #aliens_score = ((self.max_aliens - aliens_remaining)/ self.max_aliens)/100
        #return aliens_score

    def end_state(self, hasWon):
        if hasWon == True and self.training_mode == False:
            self.GAME_FONT.render_to(self.screen, (80, 350), "Victory", (0, 0, 255))
            self.GAME_FONT.render_to(self.screen, (80, 300), "Enter to Continue.", (0, 0, 255))
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                    self.done = True
                if event.type == pygame.KEYDOWN and event.key == pygame.K_a:
                    self.reset()
                    return
        elif hasWon == False and self.training_mode == False:
            self.lost = True
            self.GAME_FONT.render_to(self.screen, (80, 350), "Defeat", (0, 0, 255))
            self.GAME_FONT.render_to(self.screen, (80, 300), "Enter to Continue.", (0, 0, 255))
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                    self.done = True
                    return
                if event.type == pygame.KEYDOWN and event.key == pygame.K_a:
                        self.reset()
                        return
        elif self.training_mode == True:
            print("training instance finished")
            self.done = True
    
    def clear_arrays(self, arr):
        for a in arr:
            arr.remove(a)
            
    def reset(self):
        #Reset the clock
        self.clock = pygame.time.Clock()
        #Reset the end conditions
        self.hasWon = False
        self.done = False
        self.lost = False
        #Reinitialise all of the values containing game objects
        self.enemyShotTimer = random.uniform(0.5, 10.0)
        
        self.clear_arrays(self.enemyRockets)
        self.clear_arrays(self.rockets)
        self.clear_arrays(self.potentialShooters)
        
        self.enemyRockets = []
        self.potentialShooters = []
        self.aliens = []
        #Generate a new array of aliens and reinitialise the player
        self.generator = Generator(self)
        self.player = Invaders_Player(self, self.width/2, self.height-(self.height/100 * 10), 30)
        
        self.enemySpeed = 0.125
        if self.training_mode == True:
            self.enemySpeed = 0.083
            
        return self.get_state()
    
    def update(self):
         #Check for the victory condition (zero aliens left)
        #print("in update")
        pygame.event.get()
        if len(self.aliens) == 0:
            #If this is a genuine game, show the victory screen and await input
            self.end_state(True)
            
        if not self.lost: 
            self.player.update()
            self.player.checkCollision(self)
            
            for alien in self.aliens:
                alien.checkCollision(self)
                #If the aliens have reached the floor, end the game in defeat
                if alien.y > self.height:
                    self.end_state(False)
                    break
                        
            for rocket in self.rockets:
                rocket.update(self.rocket_speed)
            for rocket in self.enemyRockets:
                rocket.update(self.enemy_rocket_speed) 
                
        #Only check and register input if not a training game
        if self.training_mode == False:
            #Loop through pygame event to check for key presses.
            for event in pygame.event.get():
                #Only enable leaving the game for non-training builds
                if event.type == pygame.QUIT:
                    self.done = True
                    return
                #Shoot a bullet if fire rate bounds are satisfied and the space bar is held down
                if not self.lost and event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                    if self.player.fire_rate == 0.0:
                        self.shoot()

            #Get the current pressed buttons and call move functionality
            pressed = pygame.key.get_pressed()
            if pressed[pygame.K_LEFT]:
                self.move_left()
            elif pressed[pygame.K_RIGHT]:
                self.move_right()
                
        #print("update 2")
        #Control Enemy laser fire
        if self.enemyShotTimer <= 0.0:                    
            #Pick a random one of the potential shooters and fire    
            if len(self.enemyRockets) == 0:
                shooter = random.randrange(0, len(self.aliens))
                self.aliens[shooter].fire(self)
                self.enemyShotTimer = random.uniform(2.0, 6.0)
        #Otherwise iterate down the fire cool-down for the aliens.
        else:
            self.enemyShotTimer -= 0.1
        #print("update 3")
        pygame.display.flip()
        self.clock.tick(60)

    def render(self, mode='human'):      
        
        if mode == "human" and self.render_delay <= 0.0:

            #Fill the screen background black
            self.screen.fill((120,50,0))

            #print("render 1")
            #If the player is not dead, draw and check collisions
            if not self.lost: 
                self.player.draw(self)
            else:
                #If the game is just a normal game, display defeat screen and awat prompt
                self.end_state(False)

            #Render the score text to the screen
            if self.training_mode == False:
                self.GAME_FONT.render_to(self.screen, (40, 350), str(self.player.score), (0, 0, 255))

            #Update the aliens
            if not self.lost: 
                for alien in self.aliens:
                    #Render all the remaining aliens
                    alien.draw(self)
                    #If the game is still going, check collisions with the aliens
                    #if not self.lost:
                    #    alien.checkCollision(self)
                        #If the aliens have reached the floor, end the game in defeat
                    #    if alien.y > self.height:
                    #        self.end_state(False)
                    #        break

                #Update the existing rockets
                #Render the rockets in each array with a fixed speed.
                for rocket in self.rockets:
                    rocket.draw()
                for rocket in self.enemyRockets:
                    rocket.draw() 


            #Get the current pressed buttons and call move functionality
            #pressed = pygame.key.get_pressed()
            #if pressed[pygame.K_UP]:
            #    self.take_picture()
            #print("render 3")
        else:
            self.render_delay -= 0.1
            
#Auxillary classes for the Space invaders game mode
class Generator:
    def __init__(self, game):
        #Reset aliens
        game.aliens = []
        #Set attribute information for each alien including position, spacing and type
        width = 50
        margin = 60
        size = 30
        if game.training_mode == True:
            size = 20
            width = 30
            margin = 40
            
            
        alienType = 4
        #Initialise iterables
        row = 0
        row_index = 0
        print("game scale: ", game.scale)
        #Cycle through as many times as the width, heigh and margin allow, making new aliens
        for y in range(0, int(game.height/2 - width), width):
            row_index = 0
            for x in range(margin, game.width - margin, width):
                game.aliens.append(Alien(game, x, y, size,
                                         "Dependencies/Resources/alien" + str(alienType) + ".png",
                                         row, row_index, alienType, game.scale))
                row_index += 1
            #Use a new alien type for each row
            row += 1
            if alienType > 1:
                alienType -= 1
                
class Alien:
    def __init__(self, game, x, y, size, path, row, row_index, alien_type, scale):
        #Position
        self.x = x
        self.y = y
        #Game reference
        self.game = game
        #Position and type reference
        self.row = row
        self.row_index = row_index
        self.alien_type = alien_type
        
        #Misc
        self.size = size
        self.alienSprites = pygame.sprite.Group()
        self.sprite = Block((130, 240, 120), self.x, self.y, path, scale)
        self.alienSprites.add(self.sprite)
        
        #Initial move direction (denoting right)
        self.moveDirection = 1
        
        
    def draw(self, game):
        index = 0
        for sprite in self.alienSprites:
            if index > 0:
                print("sprite no: ", index)
            index += 1
            #Update horizontal position
            sprite.velocity[0] = game.enemySpeed
            if self.moveDirection > 0:
                sprite.position[0] = sprite.position[0] + sprite.velocity[0]
            else:
                sprite.position[0] = sprite.position[0] - sprite.velocity[0]
            
            #If this is the end alien, and it reaches the side, call drop and reverse for all aliens
            if sprite.position[0] >= game.width-30 and self.moveDirection == 1 \
            or sprite.position[0] <= 5 and self.moveDirection == -1:
                self.moveDirection = -self.moveDirection

                for alien in game.aliens:
                    alien.drop_and_reverse(self.moveDirection)
            
            #Update sprite position
            sprite.position[1] = sprite.position[1] + sprite.velocity[1]
            sprite.rect.x = sprite.position[0]
            sprite.rect.y = sprite.position[1]
            game.screen.blit(sprite.image, sprite.rect)
    
    def fire(self, game):
        for sprite in self.alienSprites:
            game.enemyRockets.append(Rocket(self.game, sprite.rect.x+16, sprite.rect.y))
        
    def drop_and_reverse(self, newDir):
        #Only update move direction for those who didn't detect the turn
        if self.moveDirection != newDir:
            self.moveDirection = newDir
        #Drop and update sprite position
        for sprite in self.alienSprites:
            sprite.position[1] = sprite.position[1] + 5
            
    def checkCollision(self, game):
        for rocket in game.rockets:
            if rocket.y > game.height or rocket.y < 0:
                game.rockets.remove(rocket)
                
        for sprite in self.alienSprites:
            for rocket in game.rockets:
                if(rocket.x < sprite.rect.x + self.size and
                  rocket.x > sprite.rect.x - self.size and
                  rocket.y < sprite.rect.y + self.size and
                  rocket.y > sprite.rect.y - self.size):
                    game.rockets.remove(rocket)
                    game.aliens.remove(self)
                    game.player.score += 1
                    game.current_reward += 0.1
                    if len(game.aliens) < 33:
                        game.enemySpeed = 0.25
                    if len(game.aliens) < 22:
                        game.enemySpeed = 0.5
                    if len(game.aliens) < 11:
                        game.enemySpeed = 0.75
                    if len(game.aliens) < 2:
                        game.enemySpeed = 2
                        
#Class for the rocket for both player and alien.
class Rocket:
    def __init__(self, game, x, y):
        self.x = x
        self.y = y
        self.game = game
    
    def update(self, offset):
        self.y -= offset
        
    def draw(self):
        pygame.draw.rect(self.game.screen, (255, 0, 0),
                        pygame.Rect(self.x, self.y, 2, 4))

#Default container for objects using sprites
class Block(pygame.sprite.Sprite):
    def __init__(self, color, width, height, path, scale):
        super().__init__()
        # Load the image
        self.image = pygame.image.load(path).convert()
        
        # Set our transparent color
        self.image.set_colorkey((0,0,0))
        self.rect = self.image.get_rect()
        self.rect.x = width
        self.rect.y = height
        self.position = [width, height]
        self.velocity = [0, 0]
        
class Invaders_Player: 
    def __init__(self, game, x, y, size):
        self.x = x
        self.y = y
        self.size = size
        self.game = game
        self.playerSprites = pygame.sprite.Group()
        self.sprite = Block((130, 240, 120), self.x, self.y, "Dependencies/Resources/paddle.png",
                            game.scale)
        self.playerSprites.add(self.sprite)
        self.fire_rate = 3.0
        self.score = 0
        
    def draw(self, game):
        self.playerSprites.draw(game.screen)
        
    def update(self):
        if self.fire_rate != 0.0:
            self.fire_rate -=0.1
            if self.fire_rate < 0.0:
                self.fire_rate = 0.0
                
    def checkCollision(self, game):
        if len(game.aliens) > 0:
            for rocket in game.enemyRockets:
                #pygame.draw.rect(self.game.screen, (255, 255, 0),
                #    pygame.Rect(rocket.x, rocket.y, 2, 4))
                if rocket.y > game.height or rocket.y < 0:
                    game.enemyRockets.remove(rocket)

                for sprite in self.playerSprites:
                    #pygame.draw.rect(self.game.screen, (255, 0, 0),
                    #    pygame.Rect(sprite.rect.x, sprite.rect.y, self.size, self.size))
                    #pygame.draw.line(self.game.screen, (255, 100, 100), 
                     #                (sprite.rect.x, sprite.rect.y), (sprite.rect.x, sprite.rect.y - 5))
                    if(rocket.x < sprite.rect.x + (self.size) and
                          rocket.x > sprite.rect.x and
                          rocket.y > sprite.rect.y):
                            print("collision detected", rocket.x, sprite.rect.x,  self.size)
                            game.enemyRockets.remove(rocket)
                            game.lost = True
                            

In [12]:
#pygame.display.init()
#screen = None
#Space_Invaders(230, 200, screen, None, False)

game scale:  1
player position: 115.0 180.0
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
up

render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
u

render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
u

render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
update 3
render 1
exitng


SystemExit: 